### Let's Import the Libraries Which we are going to Use

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!pip install  folium 
import folium 
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup


print('Libraries imported.')

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
Libraries imported.


### First Let's get the New York Neighborhood Data

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('New York Data downloaded!')

New York Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
ny_neighborhoods_data = newyork_data['features']

In [5]:
ny_neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
for data in ny_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhoods = ny_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
ny_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(ny_neighborhoods['Borough'].unique()),
        ny_neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Now Let's get the Toronto Data and correlate it with Geo Spatial Data

In [10]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

In [12]:
my_table = soup.find('table',{'class':'wikitable sortable'})

In [13]:
#Generate lists
A=[]
B=[]
C=[]
D=[]

for row in my_table.findAll("tr"):
#    print('row: ', len(row))
    cells = row.findAll('td')
#    states=row.findAll('th') #To store second column data
#    print('cells: ', len(cells))
#    print('states: ',len(states))
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
#        B.append(states[0].find(text=True))
        C.append(cells[1].find(text=True))
        D.append(cells[2].find(text=True))

In [14]:
t_df=pd.DataFrame(A,columns=['Postcode'])
t_df['Borough']=C
t_df['Neighborhood']=D
t_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
t_df = t_df[t_df.Borough != 'Not assigned']

In [16]:
t_df = t_df.replace('\n','', regex=True)
t_df.shape

(212, 3)

In [17]:
t_df['Neighborhood'] = np.where(t_df['Neighborhood'] == 'Not assigned', t_df['Borough'], t_df['Neighborhood'])

In [18]:
t_df = t_df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()

In [19]:
t_df.shape

(103, 3)

In [20]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data

--2018-12-06 20:21:03--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2018-12-06 20:21:04--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-06 20:21:05--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197, 107.152.24.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-06 20:21:05--  https://ibm.ent

In [21]:
geo_df = pd.read_csv('Geospatial_data.csv')

In [22]:
geo_df.index
geo_df.shape

(103, 3)

In [23]:
to_neighborhoods = pd.merge(t_df, geo_df,  how='inner', left_on=['Postcode'], right_on = ['Postal Code'])

In [24]:
to_neighborhoods.drop(columns=['Postal Code'], inplace=True)

In [25]:
to_neighborhoods.shape

(103, 5)

### Let's Map both Toronto and New York Data to check if we have the correct Data

In [26]:
ny_address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(ny_address)
ny_latitude = location.latitude
ny_longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(ny_latitude, ny_longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [27]:
to_address = 'Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(to_address)
to_latitude = location.latitude
to_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(to_latitude, to_longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [28]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(ny_neighborhoods['Latitude'], ny_neighborhoods['Longitude'], ny_neighborhoods['Borough'], ny_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [29]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[to_latitude, to_longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(to_neighborhoods['Latitude'], to_neighborhoods['Longitude'], to_neighborhoods['Borough'], to_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Let's Work with Four Squares now

### First Let's Work with Manhattan and Downtown Toronto for apples to apples comparison and reduce the data set

In [30]:
# The code was removed by Watson Studio for sharing.

In [31]:
manhattan_data = ny_neighborhoods[ny_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [32]:
ma_address = 'Manhattan, NY'

geolocator = Nominatim()
ma_location = geolocator.geocode(ma_address)
ma_latitude = ma_location.latitude
ma_longitude = ma_location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(ma_latitude, ma_longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [33]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[ma_latitude, ma_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [34]:
tcity_df = to_neighborhoods[to_neighborhoods['Borough'].str.contains("Toronto")]
#print(tcity_df.shape)
#tcity_df = toronto_df 'Islington Avenue', 'Upper Rouge'
#tcity_df = df[~df.C.str.contains("XYZ")]
temp_df = tcity_df[tcity_df.Neighborhood != 'Islington Avenue']
toronto_data = temp_df[temp_df.Neighborhood != 'Upper Rouge']

toronto_data.shape

(38, 5)

In [35]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
to_location = geolocator.geocode(address)
to_latitude = to_location.latitude
to_longitude = to_location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(to_latitude, to_longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [36]:
# create map of Toronto using latitude and longitude values
#Postcode	Borough	Neighbourhood	Latitude	Longitude
map_tcity = folium.Map(location=[to_latitude, to_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tcity)  
    
map_tcity

In [37]:
# Define Limit and radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now Let's Get all the venues in Manhattan

In [39]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


### Now Let's the venues in Downtown Toronto

In [41]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [42]:
def getVenueCount(venue_counts,venueType):
    
    #temp_val = venue_counts.loc[venue_counts['Venue Category'] == venueType].Count
    #print(venue_counts[venue_counts['Venue Category'].str.contains(venueType)].Count)
    temp_val = venue_counts[venue_counts['Venue Category'].str.contains(venueType)].Count
    #print(len(temp_val))
    myTempVal = 0
    i = 0
    for tempi in temp_val:
        myTempVal = temp_val.values[i] + myTempVal
        i = i + 1
        
    return(myTempVal)

### Let's Define our Area of Interest to Open a Business

In [43]:
my_venue_types = ['Bakery','Indian','Italian','Mexican','Thai','Hotel']

### Let's get the Venue Count for Manhattan

In [44]:
#print("Top Venues in Manhattan are: \n",manhattan_venues.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False).head(10))
manhattan_venues_temp = manhattan_venues.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)
manhattan_venues_counts = pd.DataFrame({'Venue Category':manhattan_venues_temp.index, 'Count':manhattan_venues_temp.values})
#ny_Bakery_count = getVenueCount(manhattan_venues_counts,"Bakery")
#ny_Indian_count = getVenueCount(manhattan_venues_counts,"Indian")
#ny_Italian_count = getVenueCount(manhattan_venues_counts,"Italian")
#ny_Mexican_count = getVenueCount(manhattan_venues_counts,"Mexican")
#ny_Thai_count = getVenueCount(manhattan_venues_counts,"Thai")
#ny_Hotel_count = getVenueCount(manhattan_venues_counts,"Hotel")

#print("\nBakeries in NY: ",ny_Bakery_count)
#print("Indian Restaurants in NY: ",ny_Indian_count)
#print("Italian Restaurants in NY: ",ny_Italian_count)
#print("Mexican Restaurants in NY: ",ny_Mexican_count)
#print("Thai Restaurants in NY: ",ny_Thai_count)
#print("Hotels in NY: ",ny_Hotel_count)

for venue_type in my_venue_types:
    venue_count = getVenueCount(manhattan_venues_counts,venue_type)
    print("Venue Type: \"",venue_type,"\" Number in NY: ",venue_count)

Venue Type: " Bakery " Number in NY:  76
Venue Type: " Indian " Number in NY:  33
Venue Type: " Italian " Number in NY:  134
Venue Type: " Mexican " Number in NY:  62
Venue Type: " Thai " Number in NY:  26
Venue Type: " Hotel " Number in NY:  74


### Now Let's Get the Downtown Toronto Data

In [45]:
#print("Top Venues in Toronto are: \n",toronto_venues.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False).head(10))
toronto_venues_temp = toronto_venues.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)
toronto_venues_counts = pd.DataFrame({'Venue Category':toronto_venues_temp.index, 'Count':toronto_venues_temp.values})
#to_Bakery_count = getVenueCount(toronto_venues_counts,"Bakery")
#to_Indian_count = getVenueCount(toronto_venues_counts,"Indian")
#to_Italian_count = getVenueCount(toronto_venues_counts,"Italian")
#to_Mexican_count = getVenueCount(toronto_venues_counts,"Mexican")
#to_Thai_count = getVenueCount(toronto_venues_counts,"Thai")
#to_Hotel_count = getVenueCount(toronto_venues_counts,"Hotel")



#print("\nBakeries in Toronto: ",to_Bakery_count)
#print("Indian Restaurants in Toronto: ",to_Indian_count)
#print("Italian Restaurants in Toronto: ",to_Italian_count)
#print("Mexican Restaurants in Toronto: ",to_Mexican_count)
#print("Thai Restaurants in Toronto: ",to_Thai_count)
#print("Hotels in NY: ",to_Hotel_count)

for venue_type in my_venue_types:
    venue_count = getVenueCount(toronto_venues_counts,venue_type)
    print("Venue Type: \"",venue_type,"\" Number in Toronto: ",venue_count)

Venue Type: " Bakery " Number in Toronto:  36
Venue Type: " Indian " Number in Toronto:  11
Venue Type: " Italian " Number in Toronto:  40
Venue Type: " Mexican " Number in Toronto:  14
Venue Type: " Thai " Number in Toronto:  20
Venue Type: " Hotel " Number in Toronto:  45


In [46]:
temp_val = manhattan_venues[manhattan_venues['Neighborhood'].str.contains("Chinatown")]
new_temp_val = temp_val.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)
new_new_temp_val = pd.DataFrame({'Venue Category':new_temp_val.index, 'Count':new_temp_val.values})
temp_Bakery_count = getVenueCount(new_new_temp_val,"Bakery")
print("\nBakeries in NY: ",temp_Bakery_count)

#print(new_temp_val)


Bakeries in NY:  3


In [ ]:
# define the dataframe columns
my_column_names = ['Neighborhood', 'Venue Category', 'Count'] 

### Let's create a Dataframe which has the neighborhoods and Venue Types of Interest for Manhattan

In [ ]:
# instantiate the dataframe
ny_neighborhood_counts = pd.DataFrame(columns=my_column_names)
print(ny_neighborhood_counts)
for neighborhood in manhattan_data['Neighborhood']:
    for venue_type in my_venue_types:
        temp_val = manhattan_venues[manhattan_venues['Neighborhood'].str.contains(neighborhood)]
        new_temp_val = temp_val.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)
        new_new_temp_val = pd.DataFrame({'Venue Category':new_temp_val.index, 'Count':new_temp_val.values})
        temp_Bakery_count = getVenueCount(new_new_temp_val,venue_type)
        #print(venue_type, " in ", neighborhood,":",temp_Bakery_count)
        ny_neighborhood_counts = ny_neighborhood_counts.append({'Neighborhood': neighborhood,
                                          'Venue Category': venue_type,
                                          'Count': temp_Bakery_count}, ignore_index=True)

In [ ]:
ny_neighborhood_counts.shape

### Let's create a Dataframe which has the neighborhoods and Venue Types of Interest for Downtown Toronto

In [ ]:
# instantiate the dataframe
to_neighborhood_counts = pd.DataFrame(columns=my_column_names)
print(to_neighborhood_counts)
for neighborhood in toronto_data['Neighborhood']:
    for venue_type in my_venue_types:
        temp_val = toronto_venues[toronto_venues['Neighborhood'].str.contains(neighborhood)]
        new_temp_val = temp_val.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)
        new_new_temp_val = pd.DataFrame({'Venue Category':new_temp_val.index, 'Count':new_temp_val.values})
        temp_Bakery_count = getVenueCount(new_new_temp_val,venue_type)
        #print(venue_type, " in ", neighborhood,":",temp_Bakery_count)
        to_neighborhood_counts = to_neighborhood_counts.append({'Neighborhood': neighborhood,
                                          'Venue Category': venue_type,
                                          'Count': temp_Bakery_count}, ignore_index=True)

In [ ]:
to_neighborhood_counts.shape

### We are interested in opening an Indian Restaurant

In [ ]:
target_venue_type = "Indian"

In [ ]:
my_venue_column_names = ['Neighborhood', 'Count']

In [ ]:
to_neighborhood_venue_counts = pd.DataFrame(columns=my_venue_column_names)
print(to_neighborhood_venue_counts)
for neighborhood in toronto_data['Neighborhood']:
    temp_neigh = to_neighborhood_counts[(to_neighborhood_counts['Neighborhood']==neighborhood) & (to_neighborhood_counts['Venue Category']==target_venue_type)]
    #print(temp_neigh)
    temp_val = temp_neigh[temp_neigh['Venue Category'].str.contains(target_venue_type)].Count
    myTempVal = temp_val.values[0]
    #print(neighbourhood,temp_val,myTempVal)
    #print(neighborhood,myTempVal,target_venue_type)
    to_neighborhood_venue_counts = to_neighborhood_venue_counts.append({'Neighborhood': neighborhood,
                                          'Count': myTempVal}, ignore_index=True)

In [ ]:
to_neighborhood_venue_counts.shape

In [ ]:
ny_neighborhood_counts.shape

### Let's Plot the Indian Restaurants in Manhattan
We will use red for neighborhoods which have more than one restaurant and Blue for places where there is none

In [ ]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[ma_latitude, ma_longitude], zoom_start=11)

# add markers to map
#for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    temp_neigh = ny_neighborhood_counts[(ny_neighborhood_counts['Neighborhood']==neighborhood) & (ny_neighborhood_counts['Venue Category']==target_venue_type)]
    temp_val = temp_neigh[temp_neigh['Venue Category'].str.contains(target_venue_type)].Count
    #print(temp_val)
    myTempVal = temp_val.values[0]
    #print(myTempVal)
    if(myTempVal > 0):
        myColor = 'Red'
    else:
        myColor = 'Blue'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=myColor,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Let's Plot the Indian Restaurants in Downtown Toronto
We will use red for neighborhoods which have more than one restaurant and Blue for places where there is none

In [ ]:
# create map of Toronto using latitude and longitude values
map_tcity = folium.Map(location=[to_latitude, to_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    temp_neigh = to_neighborhood_counts[(to_neighborhood_counts['Neighborhood']==neighborhood) & (to_neighborhood_counts['Venue Category']==target_venue_type)]
    temp_val = temp_neigh[temp_neigh['Venue Category'].str.contains(target_venue_type)].Count
    myTempVal = temp_val.values[0]
    if(myTempVal > 0):
        myColor = 'Red'
    else:
        myColor = 'Blue'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=myColor,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tcity)  
    
map_tcity

### Looks like we can choose ffrom a couple of neighborhoods in Toronto, while Manhattan appears to be saturated